In [12]:
import numpy as np
import random

class AntColony:
    def __init__(self, distances, n_ants, n_iterations, decay, alpha=1, beta=3):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.construct_paths()
            self.spread_pheromone(all_paths, self.decay)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            print(f"Iteration {i+1}: shortest path length = {shortest_path[1]:.2f}")
        return all_time_shortest_path

    def spread_pheromone(self, all_paths, decay):
        self.pheromone *= (1 - decay)
        for path, dist in all_paths:
            for move in path:
                self.pheromone[move] += 1.0 / dist

    def construct_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            start = random.randint(0, len(self.distances) - 1)
            path = self.construct_path(start)
            all_paths.append((path, self.path_distance(path)))
        return all_paths

    def construct_path(self, start):
        path = []
        visited = set([start])
        prev = start
        for _ in range(len(self.distances) - 1):
            move = self.select_next_city(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # return to start
        return path

    def select_next_city(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        visibility = 1 / (dist + 1e-10)
        probabilities = pheromone ** self.alpha * visibility ** self.beta
        probabilities += np.random.rand(len(probabilities)) * 0.01  # small randomness
        probabilities /= probabilities.sum()
        return np.random.choice(self.all_inds, p=probabilities)

    def path_distance(self, path):
        total = 0
        for move in path:
            total += self.distances[move]
        return total


# Example usage:
if __name__ == "__main__":
    distances = np.array([
        [0, 2, 2, 5, 7],
        [2, 0, 4, 8, 2],
        [2, 4, 0, 1, 3],
        [5, 8, 1, 0, 2],
        [7, 2, 3, 2, 0]
    ])

    ant_colony = AntColony(distances, n_ants=6, n_iterations=10, decay=0.3, alpha=1, beta=3)
    best = ant_colony.run()
    print("\nBest Path:", best[0])
    print("Shortest Distance:", best[1])

Iteration 1: shortest path length = 9.00
Iteration 2: shortest path length = 8.00
Iteration 3: shortest path length = 9.00
Iteration 4: shortest path length = 9.00
Iteration 5: shortest path length = 9.00
Iteration 6: shortest path length = 9.00
Iteration 7: shortest path length = 9.00
Iteration 8: shortest path length = 6.00
Iteration 9: shortest path length = 9.00
Iteration 10: shortest path length = 9.00

Best Path: [(3, np.int64(2)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(4)), (np.int64(4), np.int64(3)), (np.int64(3), 3)]
Shortest Distance: 6
